In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The data

In [3]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [4]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [5]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [6]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [7]:
len(train_df)

50400

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [9]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

In [10]:
# steps = 11279933
# dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/24_feb/dqn_basic_{steps}')

In [11]:
#0.92 at 11,279,933 timesteps and 1900000 episodes also at 36,144,785 timestepsc(7800000 episodes)

In [12]:
# for steps in [int(5e4), int(1e5), int(3e5), int(5e5), int(8e5), int(1e6), int(2e6)]:    
# # for steps in [int(1.1e5)]:  
#     dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/26_jan/dqn_basic_{steps}')

#### The Model

In [13]:
from stable_baselines import A2C
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.callbacks import CheckpointCallback
from modules.env import LupusEnv

Invalid MIT-MAGIC-COOKIE-1 key

In [14]:
training_env = LupusEnv(X_train, y_train)
training_env = DummyVecEnv([lambda: training_env])

model = A2C('MlpPolicy', training_env, verbose=1, seed=constants.SEED, ent_coef=0.0, vf_coef=0.5, n_cpu_tf_sess=1)
checkpoint_callback = CheckpointCallback(save_freq=500000, save_path='../models/logs/a2c', name_prefix='a2c_basic')
model.learn(total_timesteps=300000000, log_interval=500000, callback=checkpoint_callback)

# Save the trained DQN agent
model.save('../models/22_mar/a2c_sb_lupus_diagnosis')
training_env.close()

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
---------------------------------
| explained_variance | 0.159    |
| fps                | 47       |
| nupdates           | 1        |
| policy_entropy     | 3.26     |
| total_timesteps    | 5        |
| value_loss         | 0.262    |
---------------------------------
---------------------------------
| explained_variance | 0        |
| fps                | 4432     |
| nupdates           | 500000   |
| policy_entropy     | 0.00038  |
| total_timesteps    | 2500000  |
| value_loss         | 1.01     |
---------------------------------
---------------------------------
| explained_variance | 0        |
| fps                | 4642     |
| nupdates           | 1000000  |
| policy_entropy     | 0.000224 |
| total_timesteps    | 5000

---------------------------------
| explained_variance | 0        |
| fps                | 4971     |
| nupdates           | 13000000 |
| policy_entropy     | 1.64e-06 |
| total_timesteps    | 65000000 |
| value_loss         | 0.982    |
---------------------------------
---------------------------------
| explained_variance | 0        |
| fps                | 4975     |
| nupdates           | 13500000 |
| policy_entropy     | 1.64e-06 |
| total_timesteps    | 67500000 |
| value_loss         | 0.978    |
---------------------------------
---------------------------------
| explained_variance | 1.19e-07 |
| fps                | 4977     |
| nupdates           | 14000000 |
| policy_entropy     | 1.64e-06 |
| total_timesteps    | 70000000 |
| value_loss         | 0.949    |
---------------------------------
---------------------------------
| explained_variance | 1.79e-07 |
| fps                | 4977     |
| nupdates           | 14500000 |
| policy_entropy     | 1.64e-06 |
| total_timest

----------------------------------
| explained_variance | 1.19e-07  |
| fps                | 5061      |
| nupdates           | 28000000  |
| policy_entropy     | 1.03e-06  |
| total_timesteps    | 140000000 |
| value_loss         | 1.08      |
----------------------------------
----------------------------------
| explained_variance | nan       |
| fps                | 5070      |
| nupdates           | 28500000  |
| policy_entropy     | 1.03e-06  |
| total_timesteps    | 142500000 |
| value_loss         | 0.921     |
----------------------------------
----------------------------------
| explained_variance | 1.19e-07  |
| fps                | 5084      |
| nupdates           | 29000000  |
| policy_entropy     | 1.03e-06  |
| total_timesteps    | 145000000 |
| value_loss         | 0.952     |
----------------------------------
----------------------------------
| explained_variance | 1.79e-07  |
| fps                | 5097      |
| nupdates           | 29500000  |
| policy_entropy    

----------------------------------
| explained_variance | -1.19e-07 |
| fps                | 5356      |
| nupdates           | 43000000  |
| policy_entropy     | 1.02e-06  |
| total_timesteps    | 215000000 |
| value_loss         | 0.959     |
----------------------------------
----------------------------------
| explained_variance | 0         |
| fps                | 5363      |
| nupdates           | 43500000  |
| policy_entropy     | 1.02e-06  |
| total_timesteps    | 217500000 |
| value_loss         | 1.02      |
----------------------------------
----------------------------------
| explained_variance | 0         |
| fps                | 5370      |
| nupdates           | 44000000  |
| policy_entropy     | 9.26e-07  |
| total_timesteps    | 220000000 |
| value_loss         | 0.943     |
----------------------------------
----------------------------------
| explained_variance | 0         |
| fps                | 5376      |
| nupdates           | 44500000  |
| policy_entropy    

----------------------------------
| explained_variance | 0         |
| fps                | 5275      |
| nupdates           | 58000000  |
| policy_entropy     | 1.74e-06  |
| total_timesteps    | 290000000 |
| value_loss         | 1.02      |
----------------------------------
----------------------------------
| explained_variance | 1.19e-07  |
| fps                | 5274      |
| nupdates           | 58500000  |
| policy_entropy     | 2.63e-06  |
| total_timesteps    | 292500000 |
| value_loss         | 1.01      |
----------------------------------
----------------------------------
| explained_variance | nan       |
| fps                | 5272      |
| nupdates           | 59000000  |
| policy_entropy     | 2.61e-06  |
| total_timesteps    | 295000000 |
| value_loss         | 0.98      |
----------------------------------
----------------------------------
| explained_variance | 1.19e-07  |
| fps                | 5270      |
| nupdates           | 59500000  |
| policy_entropy    